In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
sns.set(style="white", context='poster')
from scipy.integrate import solve_ivp

levenberg optimization

In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
#from model_equations_separate_NC import *
from model_equations_separate_NC_store_numba import *


In [4]:

HCOLOR = '#ffc58f'
HCOLOR_light = '#ffd2a9'
PCOLOR = '#99cc99'
PCOLOR_light = '#aad5aa'

DONCOLOR = '#7285b7'
RDONCOLOR = 'black' #'#00334D'
DINCOLOR = '#bbdaff' #'#000066'

DOCCOLOR = '#cc6666'
RDOCCOLOR = 'black' #'#00334D'
DICCOLOR = '#ff9da4' #'#000066'

SHCOLOR = '#ffa98f'
SPCOLOR = '#326232'


ccnpalette = [PCOLOR, PCOLOR_light, HCOLOR, HCOLOR_light, SPCOLOR, SHCOLOR,DONCOLOR, RDONCOLOR, DINCOLOR]
ccnorder = ['Bp', 'Np', 'Bh','Nh',  'ABp', 'ABh', 'DON', 'RDON', 'DIN', ]
cccpalette = [PCOLOR, PCOLOR_light, HCOLOR, HCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
cccorder = ['Bp[C]', 'Cp', 'Bh[C]', 'Ch', 'ABp[C]', 'ABh[C]', 'DOC', 'RDOC', 'DIC']
ccxpalette = [PCOLOR, HCOLOR]
ccxorder = ['Xp', 'Xh']
hnpalette = [ HCOLOR_light, SPCOLOR, SHCOLOR, DONCOLOR, RDONCOLOR, DINCOLOR]
hnorder = [ 'Bh', 'ABp', 'ABh', 'DON', 'RDON', 'DIN']
hcpalette = [ HCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
hcorder = [ 'Bh[C]', 'ABp[C]', 'ABh[C]', 'DOC', 'RDOC', 'DIC']
pnpalette = [ PCOLOR_light, SPCOLOR, SHCOLOR, DONCOLOR, RDONCOLOR, DINCOLOR]
pnorder = [ 'Bp', 'ABp', 'ABh', 'DON', 'RDON', 'DIN']
pcpalette = [ PCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
pcorder = [ 'Bp[C]', 'ABp[C]', 'ABh[C]', 'DOC', 'RDOC', 'DIC']
sns.color_palette(ccnpalette + cccpalette, )
# 


[(0.6, 0.8, 0.6),
 (0.6666666666666666, 0.8352941176470589, 0.6666666666666666),
 (1.0, 0.7725490196078432, 0.5607843137254902),
 (1.0, 0.8235294117647058, 0.6627450980392157),
 (0.19607843137254902, 0.3843137254901961, 0.19607843137254902),
 (1.0, 0.6627450980392157, 0.5607843137254902),
 (0.4470588235294118, 0.5215686274509804, 0.7176470588235294),
 (0.0, 0.0, 0.0),
 (0.7333333333333333, 0.8549019607843137, 1.0),
 (0.6, 0.8, 0.6),
 (0.6666666666666666, 0.8352941176470589, 0.6666666666666666),
 (1.0, 0.7725490196078432, 0.5607843137254902),
 (1.0, 0.8235294117647058, 0.6627450980392157),
 (0.19607843137254902, 0.3843137254901961, 0.19607843137254902),
 (1.0, 0.6627450980392157, 0.5607843137254902),
 (0.8, 0.4, 0.4),
 (0.0, 0.0, 0.0),
 (1.0, 0.615686274509804, 0.6431372549019608)]

In [5]:

 
sns.color_palette([PCOLOR, HCOLOR, PCOLOR_light, HCOLOR_light] )


[(0.6, 0.8, 0.6),
 (1.0, 0.7725490196078432, 0.5607843137254902),
 (0.6666666666666666, 0.8352941176470589, 0.6666666666666666),
 (1.0, 0.8235294117647058, 0.6627450980392157)]

In [6]:
refdf = pd.read_excel('reference_10cc_axenic.xlsx')


In [7]:
refdf.columns

Index(['Sample', 'id', 'replicate', 'transfer', 'transfer day', 'heterotroph',
       'full name', 'Group', 't', 'day', 'FL', 'ref_Bp[N]', 'ref_Bp[C]'],
      dtype='object')

In [8]:
param_vals = get_param_vals('MIN')
param_vals = get_param_vals('OVERFLOW')
param_vals = get_param_vals('MIXOTROPH')
param_vals = get_param_vals('EXOENZYME')


In [9]:
param_vals

{'VmaxICp': 2.2315471698113204e-06,
 'VmaxICh': 0.0,
 'VmaxINp': 5.555555555555556e-06,
 'VmaxINh': 1.6666666666666667e-06,
 'VmaxOCp': 0.0,
 'VmaxOCh': 5.541666666666666e-08,
 'VmaxONp': 0.0,
 'VmaxONh': 4.277777777777778e-08,
 'KICp': 37.0,
 'KICh': 1.0,
 'KINp': 0.02,
 'KINh': 0.02,
 'KOCp': 0.03,
 'KOCh': 0.1,
 'KONp': 0.1,
 'KONh': 0.1,
 'Mp': 1.157407407407407e-06,
 'Mh': 1.157407407407407e-06,
 'gammaDp': 0.8,
 'gammaDh': 0.8,
 'Rp': 6.625,
 'Rh': 5.0,
 'QCmaxp': 10.0,
 'QCminp': 4.0,
 'QCmaxh': 10.0,
 'QCminh': 4.0,
 'Kmtbp': 3.5e-05,
 'Kmtbh': 3.5e-05,
 'bp': 0.01,
 'bh': 0.01,
 'r0p': 2.083333333333333e-06,
 'r0h': 2.083333333333333e-06,
 'KdecayDON': 2.31481481481482e-08,
 'KprodEXOp': 0.0,
 'KprodEXOh': 1.15740740740741e-06,
 'OverflowMode': 0.0,
 'Koverflowp': 0.0,
 'Koverflowh': 0.0,
 'ROSMode': 0.0,
 'KprodROSp': 0.0,
 'KprodROSh': 0.0,
 'KlossROSp': 0.0,
 'KlossROSh': 0.0,
 'KdecayROS': 0.0,
 'omegaP': 0.0,
 'omegaH': 0.0,
 'paramROSmaxD': 0.0}

In [10]:
print_params()

VmaxICp  = 2.23e-06, 1.93e-01
VmaxICh  = 0.00e+00, 0.00e+00
VmaxINp  = 5.56e-06, 4.80e-01
VmaxINh  = 1.67e-06, 1.44e-01
VmaxOCp  = 2.70e-08, 2.34e-03
VmaxOCh  = 5.54e-08, 4.79e-03
VmaxONp  = 1.09e-07, 9.41e-03
VmaxONh  = 4.28e-08, 3.70e-03
KICp  = 3.70e+01, 3.20e+06
KICh  = 1.00e+00, 8.64e+04
KINp  = 2.00e-02, 1.73e+03
KINh  = 2.00e-02, 1.73e+03
KOCp  = 3.00e-02, 2.59e+03
KOCh  = 1.00e-01, 8.64e+03
KONp  = 1.00e-01, 8.64e+03
KONh  = 1.00e-01, 8.64e+03
Mp  = 1.16e-06, 1.00e-01
Mh  = 1.16e-06, 1.00e-01
gammaDp  = 8.00e-01, 6.91e+04
gammaDh  = 8.00e-01, 6.91e+04
Rp  = 6.62e+00, 5.72e+05
Rh  = 5.00e+00, 4.32e+05
QCmaxp  = 1.00e+01, 8.64e+05
QCminp  = 4.00e+00, 3.46e+05
QCmaxh  = 1.00e+01, 8.64e+05
QCminh  = 4.00e+00, 3.46e+05
Kmtbp  = 3.50e-05, 3.02e+00
Kmtbh  = 3.50e-05, 3.02e+00
bp  = 1.00e-02, 8.64e+02
bh  = 1.00e-02, 8.64e+02
r0p  = 2.08e-06, 1.80e-01
r0h  = 2.08e-06, 1.80e-01
KdecayDON  = 2.31e-08, 2.00e-03
KprodEXOp  = 0.00e+00, 0.00e+00
KprodEXOh  = 1.16e-06, 1.00e-01
OverflowMode  

In [11]:
len(param_vals)

47

In [12]:
pro_vol, alt_vol, pro_vol**0.67, alt_vol**0.67, Qp, Qh

(0.5235987755982988,
 4.1887902047863905,
 0.6482299264066116,
 2.6109548884319835,
 8.92857142857143e-10,
 2.857142857142857e-09)

In [13]:
Qh * 14 * 1e9, Qp * 14 * 1e9, 

(40.0, 12.500000000000002)

In [14]:
print_equations()


NameError: name 'print_equations' is not defined

In [15]:
#### %%timeit -r 1 -n 1
var_names, init_vars, calc_dydt, interm_names, intermediate_func = get_main_data(param_vals, pro99_mode=False)

NameError: name 'get_main_data' is not defined

In [ ]:
len(var_names), len(init_vars),  len(interm_names), 

In [ ]:
print_dydt0(calc_dydt, var_names, init_vars)


In [ ]:
print_intermediate0(intermediate_func, interm_names, init_vars)


In [ ]:
biomass_diff0(calc_dydt, var_names,init_vars, param_vals)


In [ ]:
init_vars

In [ ]:
#   %%timeit -r 1 -n 1

sol = run_solver(calc_dydt, init_vars, days=140, t_eval=get_t_eval(maxday=140, step = 3600*4, ref_times = refdf['t']))

In [ ]:
sol

In [ ]:
df = solver2df(sol, var_names, interm_names, intermediate_func, param_vals)

In [ ]:
df.columns

In [ ]:
mdf = df.melt(id_vars=['t', 'day'])

In [ ]:
mdf.variable.unique()

In [ ]:
mdf.nsmallest(10, 'value')

In [ ]:
df.head()

In [ ]:
df.min().head(30)

In [ ]:
df.describe().T.head(30)

In [ ]:
sns.set_context('talk')
sns.relplot(data=mdf.loc[~mdf.value.isin([np.Inf, -np.Inf, ])], x='t', y='value', hue='variable',
            kind='line', 
            col='variable', col_wrap=6, facet_kws=dict(sharey=False),
            legend=False,
           ).set_titles(row_template = '{row_name}', col_template = '{col_name}')

In [ ]:
plt.show()

In [ ]:
df.columns

In [ ]:
df.loc[df.t.isin([11, 12])][['t','Bp', 'Np', 'Cp', 'Bh', 'Nh', 'Ch', 'DON', 'RDON', 'DIN', 'DOC', 'RDOC',
       'DIC',  'netDeltaNp', 'netDeltaNh',
       'netDeltaCp', 'netDeltaCh', 'store_keepNp', 'store_keepNh',
       'overflowNp', 'overflowNh', 'overflowCp', 'overflowCh', 'biomass_breakdown_for_respirationCh'
]].T


In [ ]:
refdf.columns

In [ ]:
g = sns.relplot(data=mdf.loc[mdf.variable.isin(ccnorder)], 
            x='day', y='value', hue='variable',
            kind='line', 
            lw=5, #legend=False,
            palette=ccnpalette, hue_order=ccnorder,
           ).set(ylabel='umol N/l', )
sns.scatterplot(data=refdf, x='day', y='ref_Bp', ax=g.ax, s=100, legend=False, color=PCOLOR)
#sns.scatterplot(data=refdf, x='day', y='cc Bh[N]', ax=g.ax, s=100, legend=False, color=HCOLOR)

g = sns.relplot(data=mdf.loc[mdf.variable.isin(cccorder)], 
            x='day', y='value', hue='variable',
            kind='line', 
            lw=5, #legend=False,
            palette=cccpalette, hue_order=cccorder,
           ).set(ylabel='umol C/l', )
#sns.scatterplot(data=refdf, x='day', y='cc Bp[C]', ax=g.ax, s=100, legend=False, color=PCOLOR)
#sns.scatterplot(data=refdf, x='day', y='cc Bh[C]', ax=g.ax, s=100, legend=False, color=HCOLOR)

# g = sns.relplot(data=mdf.loc[mdf.variable.isin(['Xp', 'Xh'])], 
#             x='day', y='value', hue='variable',
#             kind='line', 
#             lw=5, legend=False,
#             palette=ccxpalette, hue_order=ccxorder,
#            ).set(ylabel='cells/l')
#sns.scatterplot(data=refdf, x='day', y='cc mit9312', ax=g.ax, s=100, legend=False, color=PCOLOR)
#sns.scatterplot(data=refdf, x='day', y='cc 1A3', ax=g.ax, s=100, legend=False, color=HCOLOR)
plt.show()

In [ ]:
df.Bh.value_counts()

In [ ]:
sns.set_context('paper')
sns.relplot(data=df, x='day', y='Cp')
sns.relplot(data=df, x='day', y='Nh')
#plt.ylim((0,0.0002))
#plt.yscale('log')
plt.show()

In [ ]:
df.columns

In [ ]:
df.clip(lower=0)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
df.clip(lower=0)[ccnorder+ ['day']].plot.area(x='day', 
                                colormap=LinearSegmentedColormap.from_list("", ccnpalette) 
                               )
plt.legend(bbox_to_anchor =(1,1))
plt.show()
df.clip(lower=0)[cccorder+ ['day']].plot.area(x='day', 
                                colormap=LinearSegmentedColormap.from_list("", cccpalette) 
                               )
plt.legend(bbox_to_anchor =(1,1))
plt.show()


In [ ]:
    sfunc_list = [dBpdt, dNpdt, dCpdt, dBhdt, dNhdt, dChdt, dDONdt, dRDONdt, dDINdt, dDOCdt, dRDOCdt, dDICdt, dROSdt, dABpdt, dABhdt]
    subs_funclist = [sfunc.subs(param_vals) for sfunc in sfunc_list]
    
    #final_func = lambdify(var_list, subs_funclist, modules=['math'])
    #final_func_jit = jit(final_func, nopython=True)  
    #final_func_jit = lambdify(var_list, subs_funclist)
    #calc_dydt = lambda t, y : final_func_jit(*y)


In [ ]:
str(subs_funclist)

In [ ]:
simplify_sfunc_list = [simplify(i) for i in sfunc_list]

In [ ]:
simplify_sfunc_list